In [1]:
%load_ext autoreload
%autoreload 2

# World of Warcraft Log Toy

This notebook toys with data generated from World of Warcraft (WoW) logs.

4 helper classes were created (in the wow folder):
- **wow.log.Record**
  - *Parses and stores an entry of the log*
  - Each line may have differente structure so more classes deriving from this one may be necessary.
  - Each record also represent an in game event.
- **wow.log.Encounter**
  - *A set of records that represents a fight agains a boss in the game*
  - Each starts with a ENCOUNTER_START event and go until an ENCOUNTER_END event is found.
- **wow.query.Query**
  - *An object to help create a chain of iterators to build queries (using filters and map like functions)*
- **wow.query.Predicate**
  - *A Collection of functions that will be applied to those iterators*

There are also another 2 modules in addition to both modules containing classes above:
- **ui**
  - contain ui elements in this notebook

- **fights**
  - contains scripts to analyze each fight (only nerzhul for now)


In [2]:
# Initialize
import wow.ui as ui
import wow.helper as help
import wow.fights

from wow.query import Query, Predicate
from wow.log import Record, Encounter, EncounterReport

# (z, f) = ui.import_file()


# Create Data Structures

In [3]:
# help.unzip('WoWCombatLog-012722_214646.zip', 'wow')

In [4]:
file = r'C:\Program Files (x86)\World of Warcraft\_retail_\Logs\RaiderIOLogsArchive\WoWCombatLog-012722_214646.txt'
# file = r'C:\Program Files (x86)\World of Warcraft\_retail_\Logs\WoWCombatLog-013122_220632.txt'

(log, encounters) = Encounter.parse(file)


Encounters:  20


# Detailed Analysis

In [5]:
# Define Encounter being Analysed
encSelect = ui.pick_encounter(encounters)

In [6]:
e = encounters[encSelect.value]
e.getReport().report()


<style>
sb { color: steelblue }
o { color: Orange }
g { color: Green }
</style>

## <sb>"Remnant of Ner'zhul" (id: 2432)</sb>
- ENCOUNTER_START (log: 85255)
  - *2022-01-27 22:27:20.478000*  
- 34463 entries in **0:01:46.772000**
- ENCOUNTER_END (log: 119717)
  - *2022-01-27 22:29:07.250000*
    

None

,TimeStamp,ID,Player
0,2022-01-27 22:27:46.341000,Creature-0-4216-2450-10260-177117-0000734684,"""Thilokz-Azralon"""
1,2022-01-27 22:27:48.699000,Creature-0-4216-2450-10260-177117-0000734685,"""Spartanovix-Azralon"""
3,2022-01-27 22:28:34.587000,Creature-0-4216-2450-10260-177117-00007346AE,"""Thilokz-Azralon"""
2,2022-01-27 22:28:30.916000,Creature-0-4216-2450-10260-177117-00007346B0,"""Djøw-Azralon"""
4,2022-01-27 22:28:35.563000,Creature-0-4216-2450-10260-177117-00007346B0,"""Dii-Azralon"""
5,2022-01-27 22:28:43.512000,Creature-0-4216-2450-10260-177117-00007346B0,"""Spartanovix-Azralon"""


,ID,TimeStamp Spawned,TimeStamp Disposed,Time Elapsed,Player
0,Creature-0-4216-2450-10260-177117-0000734684,2022-01-27 22:27:30.935000,2022-01-27 22:27:46.341000,0 days 00:00:15.406000,"""Thilokz-Azralon"""
1,Creature-0-4216-2450-10260-177117-0000734685,2022-01-27 22:27:32.369000,2022-01-27 22:27:48.699000,0 days 00:00:16.330000,"""Spartanovix-Azralon"""
2,Creature-0-4216-2450-10260-177117-00007346AE,2022-01-27 22:28:13.480000,2022-01-27 22:28:34.587000,0 days 00:00:21.107000,"""Thilokz-Azralon"""
3,Creature-0-4216-2450-10260-177117-00007346B0,2022-01-27 22:28:14.978000,2022-01-27 22:28:43.512000,0 days 00:00:28.534000,"""Spartanovix-Azralon"""


In [7]:
# Full Log Report
# NerzhulReport(log).listOrbsDisposed().groupby(
#   lambda x: x[1],
#   lambda x: x[2],
#   lambda x: len(list(x))
# ).dict()
# r.listEncounters()

# Combined Logs
# r = EncounterReport(encounters)
# r.listEncounters()


In [9]:
# import humanize
# from pympler import asizeof
# humanize.naturalsize(
# )
# asizeof.asizeof(log, limit=6, stats=3, cutoff=0)


<style>
sb { color: steelblue }
o { color: Orange }
g { color: Green }
</style>

## <sb>"Remnant of Ner'zhul" (id: 2432)</sb>
- ENCOUNTER_START (log: 85255)
  - *2022-01-27 22:27:20.478000*  
- 34463 entries in **0:01:46.772000**
- ENCOUNTER_END (log: 119717)
  - *2022-01-27 22:29:07.250000*


None